In [150]:
import json
import requests
import datetime
import time
import os
import psycopg2
import pandas as pd
from urllib.request import urlopen
import json
import csv
from pprint import pprint
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
from datetime import datetime
import io
import numpy as np

In [106]:

#Use default user name as postgres ,if username root is not working

db_connection = psycopg2.connect(host='127.0.0.1',dbname="postgres", user="postgres" , password="root")



In [107]:
db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) 

cursor = db_connection.cursor()


In [108]:
# Only once you need to create the postgis extension.
#cursor.execute("CREATE DATABASE UML_PROJECT;")
db_connection.commit()

db_connection = psycopg2.connect(host='127.0.0.1',dbname="uml_project", user="postgres" , password="ashok0898")


cursor = db_connection.cursor()

In [109]:
db_connection.commit()

In [136]:
cursor.execute("DROP TABLE IF EXISTS language")
cursor.execute("DROP TABLE IF EXISTS exercisecategory")
cursor.execute("DROP TABLE IF EXISTS exercise")
cursor.execute("DROP TABLE IF EXISTS exerciseimage")
cursor.execute("DROP TABLE IF EXISTS muscle")
# Maintain the mapping of exercise and Muscle.
cursor.execute("DROP TABLE IF EXISTS map_exercise_muscle")
cursor.execute("DROP TABLE IF EXISTS equipment")
# Maintain the mapping of exercise and equipment
cursor.execute("DROP TABLE IF EXISTS map_exercise_equipment")
cursor.execute("DROP TABLE IF EXISTS ingredient")
cursor.execute("DROP TABLE IF EXISTS ingredientweightunit")

db_connection.commit()

cursor.execute("""CREATE TABLE language(
                id INTEGER,
                short_name TEXT,
                full_name TEXT);""")

cursor.execute("""CREATE TABLE exercisecategory(
                id INTEGER,
                name TEXT);""")

cursor.execute("""CREATE TABLE exercise(
                id INTEGER,
                status TEXT,
                description TEXT,
                name TEXT,
                name_original TEXT,
                creation_date TEXT,
                category INTEGER,
                language INTEGER
                );""")

cursor.execute("""CREATE TABLE exerciseimage(
                id INTEGER,
                status TEXT,
                image TEXT,
                is_main BOOLEAN,
                exercise INTEGER
                );""")

cursor.execute("""CREATE TABLE muscle(
                id INTEGER,
                name TEXT,
                is_front BOOLEAN
                );""")

cursor.execute("""CREATE TABLE map_exercise_muscle(
                ex_id INTEGER,
                mu_id INTEGER
                );""")

cursor.execute("""CREATE TABLE equipment(
                id INTEGER,
                name TEXT
                );""")

cursor.execute("""CREATE TABLE map_exercise_equipment(
                ex_id INTEGER,
                eq_id INTEGER
                );""")


cursor.execute("""CREATE TABLE ingredient(
                id INTEGER,
                status TEXT,
                name TEXT,
                energy INTEGER,
                protein TEXT,
                carbohydrates TEXT,
                carbohydrates_sugar TEXT,
                fat TEXT,
                fat_saturated TEXT,
                fibres TEXT,
                sodium TEXT,
                language INTEGER);""")

cursor.execute("""CREATE TABLE ingredientweightunit(
                id INTEGER,
                gram INTEGER,
                amount TEXT,
                ingredient INTEGER,
                unit INTEGER);""")

In [171]:
db_connection.commit()

### Insert to - language TABLE

In [138]:
url="http://wger.de/api/v2/language/?format=json"
response = urlopen(url)

# Extract the body of the reply
response_body = response.read()

# Decode the format in json format
languages = json.loads(response_body.decode("utf-8"))

df_languages = pd.DataFrame(languages['results'])

df_languages.to_csv('languages.csv',sep=',', encoding='utf-8', index=False)

# delete existing data in the table
cursor.execute("DELETE FROM language")
db_connection.commit()

# Now we will connect to the database and store language data
with open('languages.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row.
    for row in reader:
        cursor.execute( "INSERT INTO language VALUES (%s, %s, %s)", row)
db_connection.commit()
print("Data inserted successfully to language table")

Data inserted successfully to language table


### Insert to - exercisecategory TABLE

In [139]:
url="http://wger.de/api/v2/exercisecategory/?format=json"
response = urlopen(url)

# Extract the body of the reply
response_body = response.read()

# Decode the format in json format
exercise_category = json.loads(response_body.decode("utf-8"))

df_exercise_category = pd.DataFrame(exercise_category['results'])

df_exercise_category.to_csv('exercise_category.csv',sep=',', encoding='utf-8', index=False)

# delete existing data in the table
cursor.execute("DELETE FROM exercisecategory")
db_connection.commit()

# Now we will connect to the database and store exercisecategory data
with open('exercise_category.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row.
    for row in reader:
        cursor.execute( "INSERT INTO exercisecategory VALUES (%s, %s)", row)
db_connection.commit()
print("Data inserted successfully to exercisecategory table")


Data inserted successfully to exercisecategory table


### Insert to - exercise TABLE

In [101]:
url="http://wger.de/api/v2/exercise/?format=json"
response = urlopen(url)
print(url)

# Extract the body of the reply
response_body = response.read()

# Decode the format in json format
exercises = json.loads(response_body.decode("utf-8"))

exercises_dict = exercises['next']

df_exercises = pd.DataFrame(exercises['results'])
        
list_df = []
list_df.append(df_exercises)
while exercises_dict != None:
    print(exercises_dict)
    response = urlopen(exercises_dict)
    response_body = response.read()
    exercises = json.loads(response_body.decode("utf-8"))
    list_df.append(pd.DataFrame(exercises['results']))
    #df_exercises.append(pd.DataFrame(exercises['results']))
    #print(len(df_exercises))
    exercises_dict = exercises['next']
df_all_exercises = pd.concat(list_df, axis=0, join='outer', ignore_index=False)

http://wger.de/api/v2/exercise/?format=json
http://wger.de/api/v2/exercise/?format=json&page=2
http://wger.de/api/v2/exercise/?format=json&page=3
http://wger.de/api/v2/exercise/?format=json&page=4
http://wger.de/api/v2/exercise/?format=json&page=5
http://wger.de/api/v2/exercise/?format=json&page=6
http://wger.de/api/v2/exercise/?format=json&page=7
http://wger.de/api/v2/exercise/?format=json&page=8
http://wger.de/api/v2/exercise/?format=json&page=9
http://wger.de/api/v2/exercise/?format=json&page=10
http://wger.de/api/v2/exercise/?format=json&page=11
http://wger.de/api/v2/exercise/?format=json&page=12
http://wger.de/api/v2/exercise/?format=json&page=13
http://wger.de/api/v2/exercise/?format=json&page=14
http://wger.de/api/v2/exercise/?format=json&page=15
http://wger.de/api/v2/exercise/?format=json&page=16
http://wger.de/api/v2/exercise/?format=json&page=17
http://wger.de/api/v2/exercise/?format=json&page=18
http://wger.de/api/v2/exercise/?format=json&page=19
http://wger.de/api/v2/exerci

In [140]:
len(df_all_exercises)

688

In [141]:

# Saving data to csv file                    
df_all_exercises.to_csv('exercises.csv',sep=',', encoding='utf-8', index=False)

# delete existing data in the table
cursor.execute("DELETE FROM exercise")
cursor.execute("DELETE FROM map_exercise_muscle")
db_connection.commit()

# Inserting each row
for row in df_all_exercises.iterrows():
    #print(row)
    description = row[1]['description']
    name = row[1]['name']
    name_original = row[1]['name_original']

    # Incase the 's is present in the string it adds an escape character i.e. ''
    if(description != ''):
        description = description.replace("'","''")
    if(name != ''):
        name = name.replace("'","''")
    if(name_original != ''):
        name_original = name_original.replace("'","''")
        
    query = """
    INSERT into exercise(id, status, description, name,
    name_original, creation_date, category, language) 
    values(%s,'%s','%s','%s','%s','%s',%s,%s);
    """ % (row[1]['id'], row[1]['status'], description, name,
          name_original, row[1]['creation_date'], row[1]['category'],
          row[1]['language'])

    cursor.execute(query)
    
    # Inserting data to map_exercise_muscle
    muscles = row[1]['muscles']
    if len(muscles) > 0:
        ex_id = row[1]['id']
        for m in muscles:
            #print(ex_id,m)
            ex_mu_query = """
            INSERT into map_exercise_muscle(ex_id, mu_id) 
            values(%s,%s);
            """ % (ex_id,m)
            cursor.execute(ex_mu_query)

    # Inserting data to map_exercise_equipment
    equipments = row[1]['equipment']
    if len(equipments) > 0:
        ex_id = row[1]['id']
        for e in equipments:
            ex_eq_query = """
            INSERT into map_exercise_equipment(ex_id, eq_id) 
            values(%s,%s);
            """ % (ex_id,e)
            cursor.execute(ex_eq_query)

# Close the connection - Saving the changes
db_connection.commit()
print("Data inserted successfully to exercise , map_exercise_muscle and map_exercise_equipment tables")


Data inserted successfully to exercise , map_exercise_muscle and map_exercise_equipment tables


### Insert to - exerciseimage TABLE

In [110]:
url="http://wger.de/api/v2/exerciseimage/?format=json"
response = urlopen(url)
print(url)

# Extract the body of the reply
response_body = response.read()

# Decode the format in json format
exercise_images = json.loads(response_body.decode("utf-8"))

exercise_images_dict = exercise_images['next']

df_exercise_images = pd.DataFrame(exercise_images['results'])
        
list_df = []
list_df.append(df_exercise_images)
while exercise_images_dict != None:
    print(exercise_images_dict)
    response = urlopen(exercise_images_dict)
    response_body = response.read()
    exercise_images = json.loads(response_body.decode("utf-8"))
    list_df.append(pd.DataFrame(exercise_images['results']))
    exercise_images_dict = exercise_images['next']
df_all_exercise_images = pd.concat(list_df, axis=0, join='outer', ignore_index=False)

http://wger.de/api/v2/exerciseimage/?format=json
http://wger.de/api/v2/exerciseimage/?format=json&page=2
http://wger.de/api/v2/exerciseimage/?format=json&page=3
http://wger.de/api/v2/exerciseimage/?format=json&page=4
http://wger.de/api/v2/exerciseimage/?format=json&page=5
http://wger.de/api/v2/exerciseimage/?format=json&page=6
http://wger.de/api/v2/exerciseimage/?format=json&page=7
http://wger.de/api/v2/exerciseimage/?format=json&page=8
http://wger.de/api/v2/exerciseimage/?format=json&page=9
http://wger.de/api/v2/exerciseimage/?format=json&page=10
http://wger.de/api/v2/exerciseimage/?format=json&page=11


In [111]:
len(df_all_exercise_images)

204

In [142]:
# Saving data to csv file                    
df_all_exercise_images.to_csv('exercise_images.csv',sep=',', encoding='utf-8', index=False)

# delete existing data in the table
cursor.execute("DELETE FROM exerciseimage")
db_connection.commit()

# Inserting each row
for row in df_all_exercise_images.iterrows():
        
    query = """
    INSERT into exerciseimage(id, status, image, is_main, exercise) 
    values(%s,'%s','%s',%s,%s);
    """ % (row[1]['id'], row[1]['status'], row[1]['image'],
           row[1]['is_main'], row[1]['exercise'])

    #print(query)
    cursor.execute(query)
    
# Close the connection - Saving the changes
db_connection.commit()
print("Data inserted successfully to exerciseimage tables")


Data inserted successfully to exerciseimage tables


### Insert to - muscle TABLE

In [143]:
url="http://wger.de/api/v2/muscle/?format=json"
response = urlopen(url)

# Extract the body of the reply
response_body = response.read()

# Decode the format in json format
muscles = json.loads(response_body.decode("utf-8"))

df_muscles = pd.DataFrame(muscles['results'])

df_muscles.to_csv('muscles.csv',sep=',', encoding='utf-8', index=False)

# delete existing data in the table
cursor.execute("DELETE FROM muscle")
db_connection.commit()

# Now we will connect to the database and store muscles data
with open('muscles.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row.
    for row in reader:
        cursor.execute( "INSERT INTO muscle VALUES (%s, %s, %s)", row)
db_connection.commit()
print("Data inserted successfully to muscle table")


Data inserted successfully to muscle table


### Insert to - equipment TABLE

In [144]:
url="http://wger.de/api/v2/equipment/?format=json"
response = urlopen(url)

# Extract the body of the reply
response_body = response.read()

# Decode the format in json format
equipments = json.loads(response_body.decode("utf-8"))

df_equipments = pd.DataFrame(equipments['results'])

df_equipments.to_csv('equipments.csv',sep=',', encoding='utf-8', index=False)

# delete existing data in the table
cursor.execute("DELETE FROM equipment")
db_connection.commit()

# Now we will connect to the database and store equipments data
with open('equipments.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row.
    for row in reader:
        cursor.execute( "INSERT INTO equipment VALUES (%s, %s)", row)
db_connection.commit()
print("Data inserted successfully to equipment table")


Data inserted successfully to equipment table


### Insert to - ingredient TABLE

In [128]:
url="http://wger.de/api/v2/ingredient/?format=json"
response = urlopen(url)
print(url)

# Extract the body of the reply
response_body = response.read()

# Decode the format in json format
ingredients = json.loads(response_body.decode("utf-8"))

ingredients_dict = ingredients['next']

df_ingredients = pd.DataFrame(ingredients['results'])
        
list_df = []
list_df.append(df_ingredients)
while ingredients_dict != None:
    print(ingredients_dict)
    response = urlopen(ingredients_dict)
    response_body = response.read()
    ingredients = json.loads(response_body.decode("utf-8"))
    list_df.append(pd.DataFrame(ingredients['results']))
    ingredients_dict = ingredients['next']
df_all_ingredients = pd.concat(list_df, axis=0, join='outer', ignore_index=False)

http://wger.de/api/v2/ingredient/?format=json
http://wger.de/api/v2/ingredient/?format=json&page=2
http://wger.de/api/v2/ingredient/?format=json&page=3
http://wger.de/api/v2/ingredient/?format=json&page=4
http://wger.de/api/v2/ingredient/?format=json&page=5
http://wger.de/api/v2/ingredient/?format=json&page=6
http://wger.de/api/v2/ingredient/?format=json&page=7
http://wger.de/api/v2/ingredient/?format=json&page=8
http://wger.de/api/v2/ingredient/?format=json&page=9
http://wger.de/api/v2/ingredient/?format=json&page=10
http://wger.de/api/v2/ingredient/?format=json&page=11
http://wger.de/api/v2/ingredient/?format=json&page=12
http://wger.de/api/v2/ingredient/?format=json&page=13
http://wger.de/api/v2/ingredient/?format=json&page=14
http://wger.de/api/v2/ingredient/?format=json&page=15
http://wger.de/api/v2/ingredient/?format=json&page=16
http://wger.de/api/v2/ingredient/?format=json&page=17
http://wger.de/api/v2/ingredient/?format=json&page=18
http://wger.de/api/v2/ingredient/?format=jso

http://wger.de/api/v2/ingredient/?format=json&page=153
http://wger.de/api/v2/ingredient/?format=json&page=154
http://wger.de/api/v2/ingredient/?format=json&page=155
http://wger.de/api/v2/ingredient/?format=json&page=156
http://wger.de/api/v2/ingredient/?format=json&page=157
http://wger.de/api/v2/ingredient/?format=json&page=158
http://wger.de/api/v2/ingredient/?format=json&page=159
http://wger.de/api/v2/ingredient/?format=json&page=160
http://wger.de/api/v2/ingredient/?format=json&page=161
http://wger.de/api/v2/ingredient/?format=json&page=162
http://wger.de/api/v2/ingredient/?format=json&page=163
http://wger.de/api/v2/ingredient/?format=json&page=164
http://wger.de/api/v2/ingredient/?format=json&page=165
http://wger.de/api/v2/ingredient/?format=json&page=166
http://wger.de/api/v2/ingredient/?format=json&page=167
http://wger.de/api/v2/ingredient/?format=json&page=168
http://wger.de/api/v2/ingredient/?format=json&page=169
http://wger.de/api/v2/ingredient/?format=json&page=170
http://wge

http://wger.de/api/v2/ingredient/?format=json&page=302
http://wger.de/api/v2/ingredient/?format=json&page=303
http://wger.de/api/v2/ingredient/?format=json&page=304
http://wger.de/api/v2/ingredient/?format=json&page=305
http://wger.de/api/v2/ingredient/?format=json&page=306
http://wger.de/api/v2/ingredient/?format=json&page=307
http://wger.de/api/v2/ingredient/?format=json&page=308
http://wger.de/api/v2/ingredient/?format=json&page=309
http://wger.de/api/v2/ingredient/?format=json&page=310
http://wger.de/api/v2/ingredient/?format=json&page=311
http://wger.de/api/v2/ingredient/?format=json&page=312
http://wger.de/api/v2/ingredient/?format=json&page=313
http://wger.de/api/v2/ingredient/?format=json&page=314
http://wger.de/api/v2/ingredient/?format=json&page=315
http://wger.de/api/v2/ingredient/?format=json&page=316
http://wger.de/api/v2/ingredient/?format=json&page=317
http://wger.de/api/v2/ingredient/?format=json&page=318
http://wger.de/api/v2/ingredient/?format=json&page=319
http://wge

http://wger.de/api/v2/ingredient/?format=json&page=451
http://wger.de/api/v2/ingredient/?format=json&page=452
http://wger.de/api/v2/ingredient/?format=json&page=453
http://wger.de/api/v2/ingredient/?format=json&page=454
http://wger.de/api/v2/ingredient/?format=json&page=455
http://wger.de/api/v2/ingredient/?format=json&page=456
http://wger.de/api/v2/ingredient/?format=json&page=457
http://wger.de/api/v2/ingredient/?format=json&page=458
http://wger.de/api/v2/ingredient/?format=json&page=459
http://wger.de/api/v2/ingredient/?format=json&page=460
http://wger.de/api/v2/ingredient/?format=json&page=461
http://wger.de/api/v2/ingredient/?format=json&page=462


In [154]:
print(len(df_all_ingredients))
df_all_ingredients.fillna('')
df_all_ingredients_new = df_all_ingredients
df_all_ingredients_new
df_all_ingredients_new.replace(r'^\s*$', np.nan, regex=True)
df_all_ingredients_new.fillna('')

9226


,id,license_author,status,creation_date,update_date,name,energy,protein,carbohydrates,carbohydrates_sugar,fat,fat_saturated,fibres,sodium,license,language
0,9217,quintern,1,2017-02-26,2017-02-26,365 Organic Green Lentils - Raw,360,26.000,60.000,2.000,1.000,0.000,,0.000,2,2
1,9218,quintern,1,2017-02-26,2017-02-26,365 Organic Super Grains - a blend of white an...,360,12.000,66.000,0.000,5.000,0.000,,0.000,2,2
2,8304,wger.de,2,2013-04-14,2017-01-19,4-Korn Waffeln,403,10.200,80.200,,3.900,,,,1,1
3,8336,alphafitness.club,2,2017-01-19,2017-01-19,Aalbessen,24,1.000,5.000,,0.000,,,,2,6
4,8337,alphafitness.club,2,2017-01-19,2017-01-19,Aardappelen,85,2.600,18.600,,0.000,,,,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,9213,alphafitness.club,2,2017-01-19,2017-01-19,Zuurtje,380,0.000,95.000,,0.000,,,,2,6
2,9214,alphafitness.club,2,2017-01-19,2017-01-19,Zwarte bessen,40,2.000,8.000,,0.000,,,,2,6
3,433,wger.de,2,2013-04-14,2017-01-19,Zwieback,426,10.100,74.200,12.500,9.700,2.525,2.500,0.227,1,2
4,8292,wger.de,2,2013-04-14,2017-01-19,"Zwiebeln, roh",40,1.100,9.340,4.240,0.100,0.042,1.700,0.004,1,1


In [169]:
# Saving data to csv file                    
df_all_ingredients_new.to_csv('ingredients.csv',sep=',', encoding='utf-8', index=False)

# delete existing data in the table
cursor.execute("DELETE FROM ingredient")
db_connection.commit()

# Inserting each row
for row in df_all_ingredients_new.iterrows():
    fiber = row[1]['fibres']
    if fiber == None :
        fiber = ''
    if(name != ''):
        name = name.replace("'","''")
        
    query = """
    INSERT into ingredient(id, status, name, energy,
    protein, carbohydrates,carbohydrates_sugar, fat,
    fat_saturated, fibres, sodium,language) 
    values(%s,'%s','%s',%s,'%s','%s','%s','%s','%s','%s','%s',%s);
    """ % (row[1]['id'], row[1]['status'], name,row[1]['energy'],
           row[1]['protein'], row[1]['carbohydrates'], row[1]['carbohydrates_sugar'],row[1]['fat'],
           row[1]['fat_saturated'], fiber,row[1]['sodium'],row[1]['language'] )

    #print(query)
    cursor.execute(query)
    
# Close the connection - Saving the changes
db_connection.commit()
print("Data inserted successfully to ingredient table")

Data inserted successfully to ingredient table


### Insert into - ingredientweightunit TABLE

In [172]:
url="http://wger.de/api/v2/ingredientweightunit/?format=json"
response = urlopen(url)
print(url)

# Extract the body of the reply
response_body = response.read()

# Decode the format in json format
ingredientsweight = json.loads(response_body.decode("utf-8"))

ingredientsweight_dict = ingredientsweight['next']

df_ingredientsweight = pd.DataFrame(ingredientsweight['results'])
        
list_df = []
list_df.append(df_ingredientsweight)
while ingredientsweight_dict != None:
    print(ingredientsweight_dict)
    response = urlopen(ingredientsweight_dict)
    response_body = response.read()
    ingredientsweight = json.loads(response_body.decode("utf-8"))
    list_df.append(pd.DataFrame(ingredientsweight['results']))
    ingredientsweight_dict = ingredientsweight['next']
df_all_ingredientsweight = pd.concat(list_df, axis=0, join='outer', ignore_index=False)

http://wger.de/api/v2/ingredientweightunit/?format=json
http://wger.de/api/v2/ingredientweightunit/?format=json&page=2
http://wger.de/api/v2/ingredientweightunit/?format=json&page=3
http://wger.de/api/v2/ingredientweightunit/?format=json&page=4
http://wger.de/api/v2/ingredientweightunit/?format=json&page=5
http://wger.de/api/v2/ingredientweightunit/?format=json&page=6
http://wger.de/api/v2/ingredientweightunit/?format=json&page=7
http://wger.de/api/v2/ingredientweightunit/?format=json&page=8
http://wger.de/api/v2/ingredientweightunit/?format=json&page=9
http://wger.de/api/v2/ingredientweightunit/?format=json&page=10
http://wger.de/api/v2/ingredientweightunit/?format=json&page=11
http://wger.de/api/v2/ingredientweightunit/?format=json&page=12
http://wger.de/api/v2/ingredientweightunit/?format=json&page=13
http://wger.de/api/v2/ingredientweightunit/?format=json&page=14
http://wger.de/api/v2/ingredientweightunit/?format=json&page=15
http://wger.de/api/v2/ingredientweightunit/?format=json&

http://wger.de/api/v2/ingredientweightunit/?format=json&page=129
http://wger.de/api/v2/ingredientweightunit/?format=json&page=130
http://wger.de/api/v2/ingredientweightunit/?format=json&page=131
http://wger.de/api/v2/ingredientweightunit/?format=json&page=132
http://wger.de/api/v2/ingredientweightunit/?format=json&page=133
http://wger.de/api/v2/ingredientweightunit/?format=json&page=134
http://wger.de/api/v2/ingredientweightunit/?format=json&page=135
http://wger.de/api/v2/ingredientweightunit/?format=json&page=136
http://wger.de/api/v2/ingredientweightunit/?format=json&page=137
http://wger.de/api/v2/ingredientweightunit/?format=json&page=138
http://wger.de/api/v2/ingredientweightunit/?format=json&page=139
http://wger.de/api/v2/ingredientweightunit/?format=json&page=140
http://wger.de/api/v2/ingredientweightunit/?format=json&page=141
http://wger.de/api/v2/ingredientweightunit/?format=json&page=142
http://wger.de/api/v2/ingredientweightunit/?format=json&page=143
http://wger.de/api/v2/ing

http://wger.de/api/v2/ingredientweightunit/?format=json&page=256
http://wger.de/api/v2/ingredientweightunit/?format=json&page=257
http://wger.de/api/v2/ingredientweightunit/?format=json&page=258
http://wger.de/api/v2/ingredientweightunit/?format=json&page=259
http://wger.de/api/v2/ingredientweightunit/?format=json&page=260
http://wger.de/api/v2/ingredientweightunit/?format=json&page=261
http://wger.de/api/v2/ingredientweightunit/?format=json&page=262
http://wger.de/api/v2/ingredientweightunit/?format=json&page=263
http://wger.de/api/v2/ingredientweightunit/?format=json&page=264
http://wger.de/api/v2/ingredientweightunit/?format=json&page=265
http://wger.de/api/v2/ingredientweightunit/?format=json&page=266
http://wger.de/api/v2/ingredientweightunit/?format=json&page=267
http://wger.de/api/v2/ingredientweightunit/?format=json&page=268
http://wger.de/api/v2/ingredientweightunit/?format=json&page=269
http://wger.de/api/v2/ingredientweightunit/?format=json&page=270
http://wger.de/api/v2/ing

http://wger.de/api/v2/ingredientweightunit/?format=json&page=383
http://wger.de/api/v2/ingredientweightunit/?format=json&page=384
http://wger.de/api/v2/ingredientweightunit/?format=json&page=385
http://wger.de/api/v2/ingredientweightunit/?format=json&page=386
http://wger.de/api/v2/ingredientweightunit/?format=json&page=387
http://wger.de/api/v2/ingredientweightunit/?format=json&page=388
http://wger.de/api/v2/ingredientweightunit/?format=json&page=389
http://wger.de/api/v2/ingredientweightunit/?format=json&page=390
http://wger.de/api/v2/ingredientweightunit/?format=json&page=391
http://wger.de/api/v2/ingredientweightunit/?format=json&page=392
http://wger.de/api/v2/ingredientweightunit/?format=json&page=393
http://wger.de/api/v2/ingredientweightunit/?format=json&page=394
http://wger.de/api/v2/ingredientweightunit/?format=json&page=395
http://wger.de/api/v2/ingredientweightunit/?format=json&page=396
http://wger.de/api/v2/ingredientweightunit/?format=json&page=397
http://wger.de/api/v2/ing

http://wger.de/api/v2/ingredientweightunit/?format=json&page=510
http://wger.de/api/v2/ingredientweightunit/?format=json&page=511
http://wger.de/api/v2/ingredientweightunit/?format=json&page=512
http://wger.de/api/v2/ingredientweightunit/?format=json&page=513
http://wger.de/api/v2/ingredientweightunit/?format=json&page=514
http://wger.de/api/v2/ingredientweightunit/?format=json&page=515
http://wger.de/api/v2/ingredientweightunit/?format=json&page=516
http://wger.de/api/v2/ingredientweightunit/?format=json&page=517
http://wger.de/api/v2/ingredientweightunit/?format=json&page=518
http://wger.de/api/v2/ingredientweightunit/?format=json&page=519
http://wger.de/api/v2/ingredientweightunit/?format=json&page=520
http://wger.de/api/v2/ingredientweightunit/?format=json&page=521
http://wger.de/api/v2/ingredientweightunit/?format=json&page=522
http://wger.de/api/v2/ingredientweightunit/?format=json&page=523
http://wger.de/api/v2/ingredientweightunit/?format=json&page=524
http://wger.de/api/v2/ing

http://wger.de/api/v2/ingredientweightunit/?format=json&page=637
http://wger.de/api/v2/ingredientweightunit/?format=json&page=638
http://wger.de/api/v2/ingredientweightunit/?format=json&page=639
http://wger.de/api/v2/ingredientweightunit/?format=json&page=640
http://wger.de/api/v2/ingredientweightunit/?format=json&page=641
http://wger.de/api/v2/ingredientweightunit/?format=json&page=642
http://wger.de/api/v2/ingredientweightunit/?format=json&page=643
http://wger.de/api/v2/ingredientweightunit/?format=json&page=644
http://wger.de/api/v2/ingredientweightunit/?format=json&page=645
http://wger.de/api/v2/ingredientweightunit/?format=json&page=646
http://wger.de/api/v2/ingredientweightunit/?format=json&page=647
http://wger.de/api/v2/ingredientweightunit/?format=json&page=648
http://wger.de/api/v2/ingredientweightunit/?format=json&page=649
http://wger.de/api/v2/ingredientweightunit/?format=json&page=650
http://wger.de/api/v2/ingredientweightunit/?format=json&page=651
http://wger.de/api/v2/ing

In [173]:
len(df_all_ingredientsweight)

14279

In [174]:
df_all_ingredientsweight.to_csv('ingredientweightunit.csv',sep=',', encoding='utf-8', index=False)

# delete existing data in the table
cursor.execute("DELETE FROM ingredientweightunit")
db_connection.commit()

# Now we will connect to the database and store equipments data
with open('ingredientweightunit.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row.
    for row in reader:
        cursor.execute( "INSERT INTO ingredientweightunit VALUES (%s, %s, %s, %s, %s)", row)
db_connection.commit()
print("Data inserted successfully to ingredientweightunit table")


Data inserted successfully to ingredientweightunit table
